In [1]:
# Some practice for scraping ptt
# This python scapper allows you to get article title, review count and url of this article from ptt

import requests
import csv
import re
from bs4 import BeautifulSoup


# Enter the subforum which you want to scrape data from
# Subforum name is case sensitive
def subforum():
    name = input("Enter which sub forum you want to scrape data from: ")
    page = int(input("How many pages do you want: "))
    print("get data from {a}".format(a = name))
    return(name, page)


# Send http request to server
def Response(a):
    url = a
    response = requests.get(url, cookies={'over18': '1'}) 
    result = response.text
    return(result)


# Return contend of the first page, index_page, which is a bs4.BeautifulSoup object
def index_page(a,b):
    print("get index_page")
    name = a
    page = b
    url = "https://www.ptt.cc/bbs/"+name+"/index.html"
    data = Response(url)
    index = BeautifulSoup(data, 'html.parser')
    return (index)


# Create a list to store url of all scrapped pages
# This list is a index for scrapping contends
def pagelist(index,a,b):
    print("get pagelist")
    name = a
    page = b
    page_list = []
    # Store the first page
    a = index.find_all('a', attrs={'class':'btn wide'})
    page_number = a[1]['href']
    page_scrapped = "https://www.ptt.cc/bbs/"+name+"/index.html"
    page_list.append(page_scrapped)
    
    # Get page number
    # Extract page number by replace non-integer characters with NONE
    # Plus 1 page to page number for index
    fix = "/bbs/{url}/index".format(url = name)
    page_number = int(page_number.replace("{a}".format(a = fix),"").replace(".html",""))
    print("Total page number is {a}.".format(a = page_number + 1))
    
    # Get url of pages, index_page is included
    k = 0
    
    # -1 is to prevent to get extra pages
    while k < page - 1:
        page_scrapped_x = page_number - (k)
        page_scrapped_y = str(page_scrapped_x)
        page_scrapped_y = "https://www.ptt.cc/bbs/"+name+"/index"+page_scrapped_y+".html"
        page_list.append(page_scrapped_y)
        k += 1
    print("Get all url of pages ... done")
    return(page_list)
    

# Get article title
def title(a):
    print("get title")
    url = a
    print("Now get title from {a}".format(a = url))
    data = Response(url)
    soup = BeautifulSoup(data, 'html.parser')
    article_title = soup.find_all('div', attrs={'class':'title'})
    title_list=[]
    i = 0
    while i < len(article_title):
        element = article_title[i].text
        element = element.replace("\n","").replace(" ","").replace("\t","").replace(",","")
        title_list.append(element)
        i += 1
    return(title_list)


# Get article review count
def review(a):
    print("get reveiw count")
    url = a
    print("Now get reveiw count from {a}".format(a = url))
    data = Response(url)
    soup = BeautifulSoup(data, 'html.parser')
    article_review = soup.find_all('div', attrs={'class':'nrec'})
    review_list=[]
    i = 0
    while i < len(article_review):
        element = article_review[i].text
        if element == '':
            element = element.replace('','0')
        if element == '爆':
            element = '99'
        review_list.append(element)
        i += 1
    return(review_list)
        

# Get article url
# This url also serves as article id in our database
def article_url(a):
    print("get article url")
    url = a
    print("Now get reveiw url from {a}".format(a = url))
    data = Response(url)
    soup = BeautifulSoup(data, 'html.parser')
    article_link = soup.find_all('div', attrs={'class':'title'})
    link_list=[]
    id_list=[]
    i = 0
    
    # Create the first list to save link, and make it as string
    while i < len(article_link):
        element = article_link[i]
        link_list.append(str(element))
        i += 1
    
    # Create the second list to extract link
    # re module could not process data in a workflow
    
    j = 0
    while j < len(link_list):
        element = link_list[j]
        article_id = re.findall("[a-zA-Z0-9]{1}.[a-zA-Z0-9]{10}.[a-zA-Z0-9]{1}.[a-zA-Z0-9]{3}.html",element)
        id_list.append(article_id)
        j += 1
    print(id_list)    
    return(id_list)

       
# Write it into a csv file
def write(a,b):
    print("Now writing data into csv file")
    name = a
    write_list = b
    csvfile = open("{a}.csv".format(a = name),"w",newline='')
    writer = csv.writer(csvfile)
    writer.writerow(["title", "review_count", "id"])
    i = 0
    while i < len(write_list):
        csvrow = write_list[i]
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(csvrow)
        i += 1
    csvfile.close()


def main():
    print("run main")
    name,page = subforum()
    soup=index_page(name,page)
    page_list=pagelist(soup,name,page)
    print(page_list)
    i = 0
    data = []
    while i < page:
        article_title = title(page_list[i])
        review_count = review(page_list[i])
        article_id = article_url(page_list[i])
        j = 0
        while j < len(article_title):
            row = [article_title[j], review_count[j], article_id[j]]
            data.append(row)
            j += 1
        i += 1
    write(name,data)
    print("Done")
    
    
if __name__ == "__main__":
    print("initialize")
    main() 

initialize
run main
Enter which sub forum you want to scrape data from: Japan_Travel
How many pages do you want: 2
get data from Japan_Travel
get index_page
get pagelist
Total page number is 6551.
Get all url of pages ... done
['https://www.ptt.cc/bbs/Japan_Travel/index.html', 'https://www.ptt.cc/bbs/Japan_Travel/index6550.html']
get title
Now get title from https://www.ptt.cc/bbs/Japan_Travel/index.html
get reveiw count
Now get reveiw count from https://www.ptt.cc/bbs/Japan_Travel/index.html
get article url
Now get reveiw url from https://www.ptt.cc/bbs/Japan_Travel/index.html


IndexError: list index out of range